This notebook will read in multiple CSV files before cleaning and concatenating them. It then sets up a list of rules to parse through the combined df for names of each attraction in the state of California and replaces empty category names with the most common and most accurate ones. Finally, it exports the csv to be used later on.

In [2]:
import pandas as pd

In [3]:
#Both CSV's downloaded from Git repo for this project
enhanced_df = pd.read_csv(r'C:\Users\turne\OneDrive\Desktop\TripleTen Project Files\enhanced_attractions.csv')
service_stations_df = pd.read_csv(r'C:\Users\turne\OneDrive\Desktop\TripleTen Project Files\california_service_stations_cleaned.csv')
display(enhanced_df.head(), service_stations_df.head())

,index,title,year,image,image_link,state,category,decade,latitude,longitude,city
0,11697,Yosemite National Park,2024.0,NaN,NaN,CA,National Park,2020.0,37.8651,-119.5383,Yosemite Valley
1,11698,Redwood National and State Parks,2024.0,NaN,NaN,CA,National Park,2020.0,41.2132,-124.0046,Crescent City
2,11699,Death Valley National Park,2024.0,NaN,NaN,CA,National Park,2020.0,36.5323,-116.9325,Death Valley
3,11700,Joshua Tree National Park,2024.0,NaN,NaN,CA,National Park,2020.0,33.8734,-115.9010,Twentynine Palms
4,11701,Sequoia National Park,2024.0,NaN,NaN,CA,National Park,2020.0,36.4864,-118.5658,Three Rivers


,name,image_link,state,city,latitude,longitude
0,World's Largest Redwood Tree Service Station (...,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,California,Ukiah,39.150166,-123.207786
1,"Gordon Spencer gas station, 4691 Huntington Dr...",https://cdn.loc.gov/service/pnp/mrg/00000/0002...,California,South Pasadena,34.113306,-118.147829
2,"Mattoon Service Station (pre-fabricated), angl...",https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Culver City,34.021122,-118.396466
3,"Richfield Gas, Route 66, Cucamonga, California",https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Cucamonga,34.099199,-117.602210
4,"Far-Go gas pumps, Main Street, Barstow, Califo...",https://cdn.loc.gov/service/pnp/mrg/00000/0005...,California,Barstow,34.898621,-117.024431


In [4]:
#Dropping columns and renaming for clarity
enhanced_df = enhanced_df.drop(columns=['index', 'image', 'year', 'decade'], axis = 1, errors='ignore')
enhanced_df = enhanced_df.rename(columns = {'title': 'name'})
enhanced_df.head()

,name,image_link,state,category,latitude,longitude,city
0,Yosemite National Park,NaN,CA,National Park,37.8651,-119.5383,Yosemite Valley
1,Redwood National and State Parks,NaN,CA,National Park,41.2132,-124.0046,Crescent City
2,Death Valley National Park,NaN,CA,National Park,36.5323,-116.9325,Death Valley
3,Joshua Tree National Park,NaN,CA,National Park,33.8734,-115.9010,Twentynine Palms
4,Sequoia National Park,NaN,CA,National Park,36.4864,-118.5658,Three Rivers


In [5]:
#Replacing 'California' with 'CA' in the state column of service_stations_df
service_stations_df['state'] = service_stations_df['state'].replace('California', 'CA')
service_stations_df.head()

,name,image_link,state,city,latitude,longitude
0,World's Largest Redwood Tree Service Station (...,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,CA,Ukiah,39.150166,-123.207786
1,"Gordon Spencer gas station, 4691 Huntington Dr...",https://cdn.loc.gov/service/pnp/mrg/00000/0002...,CA,South Pasadena,34.113306,-118.147829
2,"Mattoon Service Station (pre-fabricated), angl...",https://cdn.loc.gov/service/pnp/mrg/00000/0003...,CA,Culver City,34.021122,-118.396466
3,"Richfield Gas, Route 66, Cucamonga, California",https://cdn.loc.gov/service/pnp/mrg/00000/0003...,CA,Cucamonga,34.099199,-117.602210
4,"Far-Go gas pumps, Main Street, Barstow, Califo...",https://cdn.loc.gov/service/pnp/mrg/00000/0005...,CA,Barstow,34.898621,-117.024431


In [6]:
#regex to remove everything after first comma in the name column, removes 'City, California' from name
service_stations_df['name'] = service_stations_df['name'].str.replace(r',.*', '', regex=True).str.strip()
service_stations_df['name'].head()

0    World's Largest Redwood Tree Service Station (...
1                           Gordon Spencer gas station
2             Mattoon Service Station (pre-fabricated)
3                                        Richfield Gas
4                                     Far-Go gas pumps
Name: name, dtype: object

In [7]:
cleaned_service_stations_df = service_stations_df.drop_duplicates(subset = 'name')
cleaned_service_stations_df.duplicated().sum()

np.int64(0)

In [8]:
#Concatenating the two DataFrames
combined_df = pd.concat([enhanced_df, cleaned_service_stations_df], ignore_index=True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        558 non-null    object 
 1   image_link  492 non-null    object 
 2   state       558 non-null    object 
 3   category    66 non-null     object 
 4   latitude    558 non-null    float64
 5   longitude   558 non-null    float64
 6   city        558 non-null    object 
dtypes: float64(2), object(5)
memory usage: 30.6+ KB


In [9]:
#Looks at tail of df where I know NaN values are in category column
combined_df.tail()

,name,image_link,state,category,latitude,longitude,city
553,Corker's Landing Rocket Gas sign,https://cdn.loc.gov/service/pnp/mrg/02100/0219...,CA,NaN,37.494657,-120.846594,Turlock
554,Foster's Old Fashion Freeze ice cream sign,https://cdn.loc.gov/service/pnp/mrg/02200/0226...,CA,NaN,35.626262,-120.690132,Paso Robles
555,Peg House,https://cdn.loc.gov/service/pnp/mrg/01900/0192...,CA,NaN,39.865707,-123.714188,Leggett
556,Stump House,https://cdn.loc.gov/service/pnp/mrg/01900/0193...,CA,NaN,40.801875,-124.170756,Eureka
557,Peg House Gas,https://cdn.loc.gov/service/pnp/mrg/05500/0551...,CA,NaN,39.865707,-123.714188,Leggett


In [10]:
#Setting up rules to categorize attractions based on their names, used ChatGPT to comb through csv for common names and categories
rules = {
    'gas|station': 'Gas Station',
    'motel|inn|hotel': 'Hotel',
    'golf|mini golf': 'Sports',
    'theater|drive-in|drive in': 'Theater',
    'sign': 'Sign',
    'park': 'Park',
    'building|structure|tower|castle|house|cabin': 'Historical Building',
    'museum': 'Museum',
    'statue|sculpture': 'Statue',
    'fountain': 'Fountain',
    'cafe|diner|restaurant': 'Restaurant',
    'store|shop': 'Retail',
    'windmill': 'Windmill',
    'lighthouse': 'Lighthouse',
    'church|chapel|mission': 'Religious Site',
    '.*': 'Other'  #Default 'Other' category for sites not within set rules
}

for pattern, category in rules.items():
    combined_df.loc[
        combined_df['category'].isna() & combined_df['name'].str.contains(pattern, case=False, na=False),
        'category'
    ] = category
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        558 non-null    object 
 1   image_link  492 non-null    object 
 2   state       558 non-null    object 
 3   category    558 non-null    object 
 4   latitude    558 non-null    float64
 5   longitude   558 non-null    float64
 6   city        558 non-null    object 
dtypes: float64(2), object(5)
memory usage: 30.6+ KB


In [11]:
#Fill NaN values in image_link with 'NO_IMAGE_AVAILABLE', may come back and upload images since we were only missing ~70
combined_df['image_link'].fillna('NO_IMAGE_AVAILABLE', inplace=True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        558 non-null    object 
 1   image_link  558 non-null    object 
 2   state       558 non-null    object 
 3   category    558 non-null    object 
 4   latitude    558 non-null    float64
 5   longitude   558 non-null    float64
 6   city        558 non-null    object 
dtypes: float64(2), object(5)
memory usage: 30.6+ KB


C:\Users\turne\AppData\Local\Temp\ipykernel_19496\112495697.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['image_link'].fillna('NO_IMAGE_AVAILABLE', inplace=True)


In [12]:
combined_df.to_csv('MT_california_attractions_data_cleaned.csv', index = False)